# Scaling, Performance, and Memory

In this notebook we will work with a multi-machine cluster operating in the cloud.  We will do performance tuning on a workflow that enables interactie visualization, and learn about how to measure and improve performance in a distributed context.  We'll make some pretty images too.


## Request Dask Cluster

There are many services to create Dask clusters in the cloud.  Today we'll use Coiled.

This should take a couple minutes.

In [ ]:
# TODO: !coiled login --token TOKEN --account coiled-training

In [ ]:
import coiled

cluster = coiled.Cluster(
    n_workers=10,
    package_sync=True,
    scheduler_port=443
)

from dask.distributed import Client, wait
client = Client(cluster)

client

## Large Scale GIS Visualization

For our application we'll visualize the taxi pickup locations in the classic NYC Taxi dataset.  

This data is available to us in Parquet format on S3.  Let's take a brief look at the first few rows and see how many rows we have in total.

### Data

In [ ]:
# Read in one year of NYC Taxi data

import dask.dataframe as dd

df = dd.read_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2009",
    storage_options={"anon": True},
)
df.head()

In [ ]:
len(df)

<img src="images/nyc-taxi-scatter.png" align="right" width="40%">

### Plotting large scale data is hard

Let's say we wanted to get a map of where taxi's dropped off passengers.  In principle we'd want something like the following:

```python
df.sample(frac=0.001).compute().plot(
    x="pickup_longitude", 
    y="pickup_latitude", 
    kind="scatter",
)
```

Even at 0.1% downsampling this is still just a big blob of blue.

We can do better.

### Datashader for large scale visualization

[Datashader](https://datashader.org/) is a Python library designed to visualize large datasets.  It also happens to build on Dask.  It renders large volumes of data with better design.

We won't go into how Datashader works in this tutorial (there are excellent resources online) for us it's just a tool to show us that we're processing our data quickly.

In [ ]:
import datashader
from datashader import transfer_functions as tf
from datashader.colors import Hot
import holoviews as hv

def render(df, x_range=(-74.1, -73.7), y_range=(40.6, 40.9)):
    # Plot
    canvas = datashader.Canvas(
        x_range=x_range,
        y_range=y_range,
    )
    agg = canvas.points(
        source=df, 
        x="dropoff_longitude", 
        y="dropoff_latitude", 
        agg=datashader.count("passenger_count"),
    )
    return datashader.transfer_functions.shade(agg, cmap=Hot, how="eq_hist")

In [ ]:
%%time

render(df)

## Let's Speed This Up
That works...technically. But it's painfully slow to render. How can we speed this up?

One of the time-consuming tasks here is fetching the data from S3. We can `.persist()` the dataframe into our cluster memory before we render the interactive plot. That should speed things up a bit.

In [ ]:
df = df.persist()
wait(df)

Let's try again.  It ho

In [ ]:
%%time

render(df)

## Interact

Now that we have this running at decent interactive speeds, let's switch Datashader to interactive mode.

Warning!  There is some spurious data, so you will likely have to zoom in quite a bit.  

TODO: 
1.  Fix aspect ratio
2.  Start user nearer NYC

In [ ]:
import hvplot.dask

def interact(df):
    return df.hvplot.scatter(
        x="dropoff_longitude", 
        y="dropoff_latitude", 
        aggregator=datashader.count("passenger_count"), 
        datashade=True, cnorm="eq_hist", cmap=Hot,
        width=600, 
        height=400,
    )

interact(df)

Play around and look at interesting bits.  

Can you spot the three airports from our last exercise?

## More Data

As we interact with our data and zoom in we find that we run out of data.  Fortunately, we have more. 

So far we've looked at the data for one year, 2009.  NYC-TLC published fine-grained location information for five years from 2009-2013.  This data is stored in parquet format in an S3 bucket at this location:

```python
"s3://coiled-datasets/dask-book/nyc-tlc/2009-2013/"
```

Read this data into a new Dask dataframe using the `dd.read_parquet` function.

- How many rides does it represent?
- How much money did passengers pay roughly?


Let's visualize this entire dataset as before

In [ ]:
%%time

render(df)

## Persist, observe dashboard

In [ ]:
# TODO: make exercise

df = df.persist()

Watch the dashboard for a couple minutes.  

What do you observe?

## Reduce dataset size in memory

Our data is too big for our cluster.  We have two options:

1.  Get a bigger cluster
2.  Reduce the size of our data

Let's be efficient, and reduce the size of our data.  There are three things that we can consider:

1.  Use better data types like `"string[pyarrow]"` for object dtypes, more compact floats and ints, and categoricals
2.  Eliminate all of the columns that we don't need
3.  Sampling

Play around, see if you can get a configuration that fits nicely into memory.  

In [ ]:
# TODO: make exercise

df = df[["dropoff_latitude", "dropoff_longitude", "passenger_count"]]

In [ ]:
df = df.persist()

In [ ]:
%%time

render(df)

## Reduce number of partitions

The actual Dask work here was fast, but it took a long time for Dask to get going.  

We suspect that this is because it was hard for Dask to upload the plan/graph across WiFi to the cluster.  

Let's reduce the amount of work that Dask has to do here by reducing the number of partitions.

### Good partition size

Partitions should generally be between 100-500 MiB, depending on the size of the worker you have, the dataset you're working with, and the computation that you're doing.

Your data started out this way, but after you reduced it it's probably a lot smaller.  Use `df.repartition` to collect your same data into fewer partitions.

In [ ]:
# TODO: make exercise

df = df.repartition(partition_size="256 MiB").persist()

In [ ]:
render(df)

In [ ]:
interact(df)

## Include pickup and dropoff locations

So far we've only been looking at one of these two datasets.  Now we'll look at both together. 

We now take all of our lessons learned to set this up for interactive scaling.  

We'll be visualizing and interacting with 1+B points now.

You don't need to do anything, just execute these cells and play at the bottom.

In [ ]:
# Read in one year of NYC Taxi data

import dask.dataframe as dd

df = dd.read_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2009",
    storage_options={"anon": True},
)


In [ ]:
df = df[["dropoff_longitude", "dropoff_latitude", "pickup_longitude", "pickup_latitude"]]

# clean data
df = df.loc[
    (df.dropoff_longitude > -74.1) & (df.dropoff_longitude < -73.7) & 
    (df.dropoff_latitude > 40.6) & (df.dropoff_latitude < 40.9) &
    (df.pickup_longitude > -74.1) & (df.pickup_longitude < -73.7) &
    (df.pickup_latitude > 40.6) & (df.pickup_latitude < 40.9)
]


In [ ]:
import pandas as pd


df_dropoff = df[["dropoff_longitude", "dropoff_latitude"]]
df_dropoff["journey_type"] = "dropoff"
df_dropoff = df_dropoff.rename(columns={'dropoff_longitude': 'long', 'dropoff_latitude': 'lat'})
df_pickup = df[["pickup_longitude", "pickup_latitude"]]
df_pickup["journey_type"] = "pickup"
df_pickup = df_pickup.rename(columns={'pickup_longitude': 'long', 'pickup_latitude': 'lat'})
df = dd.concat([df_dropoff, df_pickup])

pickup_dropoff = pd.CategoricalDtype(categories=["pickup", "dropoff"])
df = df.astype({"journey_type": pickup_dropoff})

df = df.repartition(partition_size="256Mib").persist()

In [ ]:
import datashader
import hvplot.dask
import holoviews as hv
hv.extension('bokeh')

color_key = {'pickup': "#e41a1c", 'dropoff': "#377eb8"}

df_plot.hvplot.scatter(
    x="long", 
    y="lat", 
    aggregator=datashader.by("journey_type"), 
    datashade=True, 
    cnorm="eq_hist",
    width=700,
    aspect=1.33, 
    color_key=color_key
)